<a href="https://colab.research.google.com/github/GRACOPORDEUS/atividades_mestrado/blob/main/1107055_220519_algoritmos_gulosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import sys

In [ ]:
## Ler os arquivos de entrada

# Código para importar arquivos do Drive
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/Algoritmos/"
file1 = "dij10.txt"
file2 = "dij20.txt"
file3 = "dij40.txt"
file4 = "dij50.txt"

def readInputs(file):
  arquivo = open(path+file)
  linhas  = arquivo.readlines()

  vertices = (int (linhas[0]))
  shape = (vertices, vertices)
  arestas = np.zeros(shape, dtype=int)
  
  contadorLinha = -1
  contadorColuna = 0

  for linha in linhas:
    if(contadorLinha == -1): # workaround para evitar a primeira linha do arquivo (que já foi lida)
      contadorLinha += 1
    else:
      contadorColuna = contadorLinha + 1
      for i in linha.split():
        arestas[contadorLinha, contadorColuna] = (int(i))
        arestas[contadorColuna, contadorLinha] = (int(i))
        contadorColuna += 1
      contadorLinha += 1

  return vertices, arestas

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Criando a classe do Grafo com os vetores auxiliares e a matrix de referência
class Grafo:

	# Função de inicialização do Grafo
	def __init__(self, vertices):
		self.vertices = vertices # Número de vertices
		
		self.matriz = [] # matriz de adjacências que representa o grafo
		self.arestas = [] # array [u, v, w] com par de vértices e peso da aresta entre eles
		
		self.pesos = [] # array auxiliar com os pesos entre as arestas (similar à estrutura "vertices[2]")
		self.pais = [] # array auxiliar com o índice do pai do vértice i
		self.rank = [] # array auxiliar com o rank do vértice i
		self.flag = [] # array auxiliar indicando se um nó já foi considerado ou não no caminho

	# Função para adicionar uma aresta ao grafo
	def adicionarAresta(self, u, v, w):
		self.arestas.append([u, v, w])

	# Função para encontrar o pai (raiz) de um vértice recursivamente
	def encontrarPai(self, i):
		if self.pais[i] == i:
			return i
		return self.encontrarPai(self.pais[i])

	# Função que UNE duas árvores através de sua raiz
	def union(self, x, y):
		raizX = self.encontrarPai(x)
		raizY = self.encontrarPai(y)

		# Agrega a menor árvore (por rank) na maior árvore pelo nó raiz
		if self.rank[raizX] < self.rank[raizY]:
			self.pais[raizX] = raizY
		elif self.rank[raizX] > self.rank[raizY]:
			self.pais[raizY] = raizX

		# Caso ambos sejam iguais, escolhe um como pai e incrementa seu rank
		else:
			self.pais[raizY] = raizX
	
	

	# Função auxiliar para encontrar o vértice com menor distância que não está no menor caminho
	def extractMin(self):

		min = sys.maxsize # inicializa com "infinito"
		min_index = 0 # vertice referente ao menor

		for v in range(self.vertices):
			if self.pesos[v] < min and self.flag[v] == False:
				min = self.pesos[v]
				min_index = v

		return min_index

### Implementação da Solução Gulosa KRUSKAL MST

In [ ]:
# Função Kruskal para Árvore Mínima Espalhada (MST - Minimum Spanned Tree)
def CaminhoMinimoKruskal(grafo):

		arvore = [] # Arvore resultado final
		i = 0 # interação nos vértices do grafo
		arestas = 0 # interação até encontrarmos o caminho mínimo

		# Passo 1: Ordenar o atributo Graph pelas Arestas mais leves
		grafo.arestas = sorted(grafo.arestas, key=lambda item: item[2]) # O peso da aresta é o terceiro elemento do vetor grafo

		# Re-inicializando os arrays de suporte: Pais e Rank dos vértices
		grafo.pais = []
		grafo.rank = []
		for vertice in range(grafo.vertices):
			grafo.pais.append(vertice) # Cada vértice inicia como seu próprio pai
			grafo.rank.append(0) # cada vértice inicia com o rank zero (sem filhos)

		# O número de arestas para o menor caminho é o número de vértices-1
		while arestas < grafo.vertices - 1:

			# Para cada par de vértices e sua aresta,
			# vamos encontrar a primeira ARESTA SEGURA para adicionar à árvore
			u, v, w = grafo.arestas[i]
			x = grafo.encontrarPai(u)
			y = grafo.encontrarPai(v)
			i = i + 1

			# Se as raizes forem diferentes, podemos incluir na árvore de forma segura
			# Qualquer outro caso pode ser descartado, e selecionamos o próximo elemento
			if x != y:
				arvore.append([u, v, w])
				grafo.union(x, y) #chamada à função de unir 2 árvores pela raiz
				arestas = arestas + 1

		# Agora com a árvore pronta, podemos calcular o Caminho Mínimo
		caminho = 0
		for u, v, w in arvore:
			caminho += w
		
		return caminho

In [ ]:
# Chamada Principal
path = "/content/drive/MyDrive/Colab Notebooks/Algoritmos/"
file = "dij10.txt"

# Leitura dos inputs no arquivo
vertices, arestas = readInputs(file1)

# Inicialização do Grafo com base nos inputs
grafo = Grafo(vertices)
#grafo.grafo = arestas
for i in range(vertices):
	for j in range(vertices):
		grafo.adicionarAresta(i, j, arestas[i,j])

#print(grafo.grafo)

# Chamada à função de cálculo do caminho mínimo
caminhoMinimo = CaminhoMinimoKruskal(grafo)
print("Caminho Mínimo KRUSKAL no arquivo", file1, ":", caminhoMinimo)

Caminho Mínimo KRUSKAL no arquivo dij10.txt : 7072


### Implementação da Solução Gulosa PRIM

In [ ]:
# Função PRIM para Árvore Mínima Espalhada (MST - Minimum Spanned Tree)
def CaminhoMinimoPrim(grafo):
 
	grafo.pesos = [sys.maxsize] * grafo.vertices # inicia os pesos com 'infinito'
	grafo.pais = [None] * grafo.vertices # Array com os pais inicia vazio
	grafo.flag = [False] * grafo.vertices # vetor auxiliar acompanha se já foi incluído na arvore ou não

	# Escolhendo um vertice para iniciar, setando seu valor como zero
	grafo.pesos[0] = 0 # será o menor
	grafo.pais[0] = -1 # será o raiz
    
	for count in range(grafo.vertices):
		
		u = grafo.extractMin() # Encontra o menor vértice dentre os não incluídos
		
		grafo.flag[u] = True # marca como incluído na arvore mínima
    
		# Relaxando os nós adjacentes para identificar o próximo a ser visitado
		for v in range(vertices):
				# grafo.arestas[u][v] será maior que zero se forem adjacentes
				# auxIncluido[v] é falso somente para nós não incluídos na arvore mínima
        # Além disso, só troca se o peso for menor do que a Aresta já existente entre os 2 vértices
			if grafo.matriz[u][v] > 0 and grafo.flag[v] == False and grafo.pesos[v] > grafo.matriz[u][v]:
					grafo.pesos[v] = grafo.matriz[u][v]
					grafo.pais[v] = u

  # Para finalizar, contar os pesos das arestas consideradas
	pesoTotal = 0
	for i in range(1, grafo.vertices):
		pesoTotal += grafo.matriz[i][grafo.pais[i]]
	return pesoTotal

In [ ]:
# Chamada Principal

# Leitura dos inputs no arquivo
vertices, arestas = readInputs(file2)

# Inicialização do Grafo com base nos inputs
grafo2 = Grafo(vertices)
grafo2.matriz = arestas

# Chamada à função de cálculo do caminho mínimo
caminhoMinimo = CaminhoMinimoPrim(grafo2)
print("Caminho Mínimo PRIM no arquivo", file2, ":", caminhoMinimo)

Caminho Mínimo PRIM no arquivo dij20.txt : 15238


### Implementação da Solução Gulosa DIJKSTRA

In [ ]:
def Dijkstra(grafo):

    grafo.pesos = [sys.maxsize] * grafo.vertices # inicia os pesos com 'infinito'
    grafo.flag = [False] * grafo.vertices # vetor auxiliar acompanha se já foi incluído na arvore ou não

    grafo.pesos[0] = 0

    for cout in range(grafo.vertices):
      u = grafo.extractMin()
      grafo.flag[u] = True

      for v in range(grafo.vertices):
        if (grafo.matriz[u][v] > 0 and grafo.flag[v] == False and grafo.pesos[v] > grafo.pesos[u] + grafo.matriz[u][v]):
          grafo.pesos[v] = grafo.pesos[u] + grafo.matriz[u][v]
      
    return grafo.pesos[grafo.vertices - 1]

In [ ]:
# Chamada Principal

# Leitura dos inputs no arquivo
vertices, arestas = readInputs(file4)

# Inicialização do Grafo com base nos inputs
grafo3 = Grafo(vertices)
grafo3.matriz = arestas

# Chamada à função de cálculo do caminho mínimo
caminhoMinimo = Dijkstra(grafo3)
print("Caminho Mínimo DIJKSTRA no arquivo", file4, ":", caminhoMinimo)

Caminho Mínimo DIJKSTRA no arquivo dij50.txt : 6764
